In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
%matplotlib inline

import glob, os
from pathlib import *
import json

In [2]:
pwd

'/home/lettucestick/github/srs_spotify_bsns/notebooks'

### Getting paths for data

Here, I build structure into the data to make each file accessible and easier to manipulate using pathlib.  


In [3]:
# using pathlib
BASE_DIR = Path('../').resolve() 
DATA_DIR = BASE_DIR / "data"
NB_DIR = BASE_DIR / "notebooks"

In [8]:
file_list = []

os.chdir(DATA_DIR)
for f in glob.glob("*.json"):
    file_list.append(f)

file_list.sort()

In [9]:
# viewing current list contents
for i,f in enumerate(file_list):
    print(i,f)

0 FamilyPlan.json
1 Follow.json
2 KidsAccount.json
3 Payments.json
4 Playlist1.json
5 SearchQueries.json
6 StreamingHistory0.json
7 StreamingHistory1.json
8 StreamingHistory2.json
9 Userdata.json
10 YourLibrary.json


In [18]:
# opening test json file
with open(file_list[2]) as data_file:
    data = json.load(data_file)

len(data), data

JSONDecodeError: Expecting value: line 1 column 1 (char 0)